<a href="https://colab.research.google.com/github/ProgramasParaFisicaDeParticulas/lecturadecvsopendatacvsfiles-CarlosRHM/blob/main/LeyendoCSVDelOpendataDelCMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Algunos de los archivos para educación del CMS con datos abiertos están en un formato CSV. Este formato es un formato de texto sencillo en donde cada evento se guarda en columnas y cada columna correspende a una variable cinemática reconstruida con el detector. Se puede encontrar un ejemplo de esta liga
https://opendata.cern.ch/record/5208.
En partícular estos datos presentan las variables cinemáticas de dos muones producto final de la colisión protón-protón. Las variables se pueden determinar usando los cuadrimomentos de las partículas (https://es.wikipedia.org/wiki/Cuadrimomento) 

Usamos panda para leer con python el archivo CSV. 

In [13]:
#panda nos permite manipular el archivo csv https://pandas.pydata.org
import pandas as pd
#numpy nos permite hacer operaciones usando vectores https://numpy.org
import numpy as np
#plotly nos permite crear graficas interactivas https://plotly.com/python/
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go
#usamos la liga que está directamente en la página. 
particles = pd.read_csv('https://opendata.cern.ch/record/5208/files/Zmumu.csv',delimiter=',')
particles


,Run,Event,pt1,eta1,phi1,Q1,dxy1,iso1,pt2,eta2,phi2,Q2,dxy2,iso2
0,165617,74969122,54.7055,-0.4324,2.5742,1,-0.0745,0.4999,34.2464,-0.9885,-0.4987,-1,0.0712,3.4221
1,165617,75138253,24.5872,-2.0522,2.8666,-1,-0.0554,0.0000,28.5389,0.3852,-1.9912,1,0.0515,0.0000
2,165617,75887636,31.7386,-2.2595,-1.3323,-1,0.0879,0.0000,30.2344,-0.4684,1.8833,1,-0.0876,0.0000
3,165617,75779415,39.7394,-0.7123,-0.3123,1,0.0585,0.0000,48.2790,-0.1956,2.9703,-1,-0.0492,0.0000
4,165617,75098104,41.2998,-0.1571,-3.0408,1,-0.0305,1.2280,43.4508,0.5910,-0.0428,-1,0.0442,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,173692,1245050481,8.9721,-1.6158,-2.7176,1,-0.0294,8.7183,13.5266,1.8709,2.7911,1,-0.0689,0.4202
9996,173692,1244979327,39.4543,-1.1310,-2.0568,1,0.0417,0.0000,47.7209,-0.1834,1.7965,-1,-0.0794,0.0000
9997,173692,1245305163,40.6034,-1.5051,-1.6758,-1,0.0635,0.2012,44.4382,-1.0010,1.5380,1,-0.0671,1.1499
9998,173692,1245041468,37.2853,1.0024,-2.1486,1,0.0361,0.0000,41.0945,-0.1200,0.8850,-1,-0.0291,0.0000


Los nombres de las columnas se refieren a las variables que describen a llos dos muones. Corrida (Etiqueta del haz), Número de evento (colisión), Momento Transverso del primer muon, Eta del primer muón, Phi del primer muón, Carga del primer muón, distancia al vertice, lo mismo para el segundo muon y finalmente la variable de aislamiento que se relaciona con la energía de las trazas que rodean al muon. 

In [21]:
#función para reconstruir la masa invariante de las dos partículas. Usamos la definición encontrada https://en.wikipedia.org/wiki/Invariant_mass
def mtransversa(pt1,pt2,eta1,eta2,phi1,phi2):
  return np.sqrt(2*pt1*pt2*(np.cosh(eta2-eta1)-np.cos(phi1-phi2)))

#Se usan los nombres de las columnas del archivo como parámetros para usar la función que calcula la masa transversa. 
mtransv=mtransversa(particles['pt1'],particles['pt2'],particles['eta1'],particles['eta2'],particles['phi1'],particles['phi2'])

#Se crea y se grafica el histograma. 
fig_5=px.histogram(mtransv,mtransv,nbins=1000)
fig_5.update_layout(title=r"$\text{Masa invariante} \ M, \ \text{%d bins}$" %1000,
                    xaxis_title=r"$GeV$",yaxis_title="Eventos")
fig_5.show()

Verificaion de las cargas de los muones.

Para el boson vectorial Z, se sabe carga es cero. Estos datos estan filtrados con decaimientos a dos candidatos a muones, por lo cual, si decae el boson Z a 2 muones, por conservacion de carga, la suma de las cargas de los muones es cero.

Esta es una verificacion con el fin de encontrar algun evento donde la carga se diferente a cero, y así decidir si se trata de un decaimiento de Z o no.

In [15]:
df = particles['Q1']+particles['Q2']
fig = px.scatter(y=df,x=particles['Event'],marginal_y="histogram",
                 labels={'x':'Evento','y':'Carga'})
fig.show()

El Parametro de impacto en el plano transversal respecto a el vertice del muon

El requisito típico utilizado en los análisis de muones para suprimir el fondo de muones cósmicos es |dxy| < 2mm. https://arxiv.org/pdf/1206.4071.pdf pagina 38

Entonces podemos buscar y descartar los enevtos en los cuales el parametro de imacto con el plano transversal sea mayor a 2mm, la escala de los datos proporcionados está en cm 

In [52]:
df = abs(particles['dxy1'])
df2 = abs(particles['dxy2'])
y0 = np.ones(len(particles['Event']))*0.2 
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(y=df, x=particles['Event'],
                    mode='markers',
                    name='|dxy1|'))
fig.add_trace(go.Scatter(y=df2, x=particles['Event'],
                    mode='markers',
                    name='|dxy2|'))
fig.add_trace(go.Scatter(x=particles['Event'], y=y0,
                    mode='lines',
                    name='dxy max'))

fig.show()

Pseudorapidez contra el angulo azimutal

Puede ayudar a saber si hay alguna preferencia de los mouones a seguir una trayectoria precisa, otra forma de interprestarlo es que al ser tantos eventos, y como la trayectoria despues de la colision es aleatroia, si se nota un hueco el la grafica de dispersion, se deduce que hay un problema con los detectores en ese punto, ya que se espera que las trayectorias no tengan una direccion preferente.

In [17]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=particles['eta1'], x=particles['phi1'],
                    mode='markers',
                    name='eta1'))
fig.add_trace(go.Scatter(y=particles['eta2'], x=particles['phi2'],
                    mode='markers',
                    name='eta2'))
fig.show()

El momento

Dado que las partículas cuando se mueven en el acelerador, tienen un momento solo en direccion del as de partículas, quiere decir que el momento transversal es cero, por conservación de momento, el momento transverso total de los muones, de forma ideal, debe ser cero, de los datos, el momento transversal, es el modulo del momento transverso, entonces se puede encontrar un momento perdido transveral de los muones haciendo su resta. 

con un histograma de este momento, dirá la frecuencia de el valor del momento, se busca que la mayoría de los datos tengan una perdida de momento pequeña, de lo contrario, se sospecha que el momento perdido es debido a otro tipo de partículas que no son muones tales como los neutrinos, electrones, o ruido, o que los muones encontrados por los detectores no vienen presisamente del decaimiento de Z.

In [29]:
df = abs(particles['pt1']-particles['pt2'])
fig = px.histogram(df,df,nbins=1000)
fig.update_layout(title=r"$\text{Momento Perdido}, \ \text{%d bins}$" %1000,
                    xaxis_title="Missing Et GeV",yaxis_title="Eventos")
fig.show()